In [ ]:
# Building the Bayesian linear regression model
with pm.Model() as model:
    # Mutable data container for predictor variables (used to update inputs during forecasting)
    X_shared = pm.MutableData("X", X_train_scaled)

    # Prior distribution for the intercept term (baseline sales level)
    intercept = pm.Normal("intercept", mu=1500, sigma=500)

    # Prior distributions for the regression coefficients (effect sizes of predictors)
    beta_promo = pm.Normal("beta_promo", mu=0, sigma=10)
    beta_holiday = pm.Normal("beta_holiday", mu=0, sigma=10)
    beta_season = pm.Normal("beta_season", mu=0, sigma=10)

    # Prior for the standard deviation of residuals (model uncertainty)
    sigma = pm.HalfNormal("sigma", sigma=100)

    # Expected value of the outcome variable (predicted sales)
    mu = pm.Deterministic(
        "mu",
        intercept
        + beta_promo * X_shared[:, 0]
        + beta_holiday * X_shared[:, 1]
        + beta_season * X_shared[:, 2]
    )  # Linear combination of predictors

    # Likelihood function for observed data
    y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=y_train)

    # Drawing posterior samples using MCMC
    trace = pm.sample(
        2000,              # Number of posterior samples
        tune=1000,         # Number of tuning (burn-in) steps
        target_accept=0.95,# Target acceptance rate for sampler stability
        chains=4,          # Number of MCMC chains
        cores=4,           # CPU cores used for parallel sampling
        return_inferencedata=True
    )

# --- Convergence Diagnostics ---
# Using traceplots to check if MCMC chains have mixed well (no divergence, good overlap) focusing on key parameters.

az.plot_trace(
    trace,
    var_names=["intercept", "beta_promo", "beta_holiday", "beta_season", "sigma"],
    compact=True
)
plt.tight_layout()
plt.show()

# --- Posterior Summary ---
print(
    az.summary(
        trace,
        var_names=["intercept", "beta_promo", "beta_holiday", "beta_season", "sigma"],
        round_to=2
    )
)

# --- Posterior Predictive Check (Training Set) ---
# Compare simulated (model-predicted) sales vs actual sales to assess model fit

with model:
    ppc_train = pm.sample_posterior_predictive(trace, random_seed=42)

az.plot_ppc(ppc_train)
plt.title("Posterior Predictive Check (Training Set)")
plt.show()

# Predicting on test set
with model:
    pm.set_data({"X": X_test_scaled})
    ppc_test = pm.sample_posterior_predictive(trace, var_names=["mu"], random_seed=42)

mu_test_samples = ppc_test.posterior_predictive["mu"]
mu_test_mean = mu_test_samples.stack(sample=("chain", "draw")).mean("sample").values

# --- Performance metrics ---
test_metrics = {
    "MAE": mean_absolute_error(y_test, mu_test_mean),
    "RMSE": mean_squared_error(y_test, mu_test_mean, squared=False),
    "R²": r2_score(y_test, mu_test_mean)
}

print("\nTest set predictions vs actual:")
print(pd.DataFrame({
    "Date": test_df["date"].values,
    "Actual": y_test.values,
    "Predicted": mu_test_mean
}))

print("\nPerformance Metrics:")
for k, v in test_metrics.items():
    print(f"{k}: {v:.2f}")

# --- Visual Comparison: Actual vs Predicted Sales ---
plt.figure(figsize=(8, 5))
plt.plot(test_df["date"], y_test.values, marker='o', label="Actual Sales", linewidth=2)
plt.plot(test_df["date"], mu_test_mean, marker='s', linestyle='--', label="Predicted Sales", linewidth=2)
plt.xlabel("Date")
plt.ylabel("Sales in Crates")
plt.title("Actual vs Predicted Coca-Cola Sales (Test Set)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
